In [4]:
import pandas as pd
import tsfel
import glob
import os
from sklearn.preprocessing import MinMaxScaler

#df01 = pd.read_csv(r"Stiksi IDV 14 dan Tuning TC 10/XMEAS.csv")
#df01.columns = ["XMEAS 1", "XMEAS 2", "XMEAS 3", "XMEAS 4","XMEAS 5", "XMEAS 6", "XMEAS 7", "XMEAS 8",
                #"XMEAS 9", "XMEAS 10", "XMEAS 11", "XMEAS 12","XMEAS 13", "XMEAS 14", "XMEAS 15", "XMEAS 16",
                #"XMEAS 17", "XMEAS 18", "XMEAS 19", "XMEAS 20","XMEAS 21", "XMEAS 22", "XMEAS 23", "XMEAS 24",
                #"XMEAS 25", "XMEAS 26", "XMEAS 27", "XMEAS 28","XMEAS 29", "XMEAS 30", "XMEAS 31", "XMEAS 32",
                #"XMEAS 33", "XMEAS 34", "XMEAS 35", "XMEAS 36","XMEAS 37", "XMEAS 38", "XMEAS 39", "XMEAS 40","XMEAS 41"]

df01 = pd.read_csv("Stiksi IDV 14 dan Eksternal IDV 4/me03.csv")
df01.columns = ["XMEAS 9", "XMEAS 10", "XMEAS 11", "XMEAS 12"]
df02 = pd.read_csv("Stiksi IDV 14 dan Eksternal IDV 4/me06.csv")
df02.columns = ["XMEAS 21", "XMEAS 22", "XMEAS 23", "XMEAS 24"]

frame = [df01,df02]                
df = pd.concat(frame, axis=1)
df.drop(labels=range(41400,49608), axis=0, inplace=True)
df_fix = df[["XMEAS 9", "XMEAS 21"]]

MM = MinMaxScaler(feature_range=(0, 1))
dfn = MM.fit_transform(df_fix)
dfn = pd.DataFrame(dfn)

# Membuat perintah segmentasi, ekstraksi fitur, dan pelabelan
def preprocessing(XMEAS, amount, length, older):
    #kolom  : kolom XMEAS yang dipilih
    #amount : jumlah subset data hasil segmentasi
    #length : ukuran jendela
    #older  : urutan XMEAS yang digunakan
    cgf_file = tsfel.get_features_by_domain() # Penentuan domain TSFEL
    df_ef = dfn.iloc[:,XMEAS].values # Penentuan XMEAS yang digunakan
    DF = pd.DataFrame() # Pembuatan Data Frame
    for i in range (0, amount, 1): # Melakukan segmentasi data
        batas_bawah = i*length # Penentuan batas bawah per subdata
        segmentasi  = df_ef[batas_bawah:(batas_bawah + length)] # Pembagian subdata
        frame = [] # List kosong untuk pembuatan frame baru
        for j in segmentasi:
            frame.append(j) # Memasukkan pembagian subdata ke dalam frame
        DF["frame%.0d" % i] = frame # Hasil segmentasi data
    for n in range(0, len(DF.columns), 1): # Melakukan ekstraksi fitur
        extract = tsfel.time_series_features_extractor(cgf_file, DF["frame%.0d" % n], fs=100,verbose=0)
        # Penyimpanan hasil ekstraksi
        extract.to_csv(r"Hasil Ekstraksi Data/%.0d Stiksi Eksternal IDV 4/FrameJamak%.0f.csv" %(length, (n+amount*(older+6))), index=False)

# Mencetak Hasil
kolom=[0,1] # 0 saja jik hanya ingin XMEAS 9
for hasil in range(0, len(kolom)):
    preprocessing(kolom[hasil],103, 400, hasil)
        
# Membaca file yang akan digabungkan
joined_files = os.path.join("Hasil Ekstraksi Data/400 Stiksi Eksternal IDV 4", "Frame*.csv")
# Menggabungkan beberapa file menjadi 1 file
joined_list = glob.glob(joined_files) 
df_final = pd.concat(map(pd.read_csv, joined_list), ignore_index=True) 
df_final["Label"] = "3" # Pelabelan Subdata
# Menyimpan hasil penggabungan data menjadi 1 file
df_final.to_csv(r'Data 4 Stiksi Eksternal IDV 4.csv')  

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3852\2041249354.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF["frame%.0d" % i] = frame # Hasil segmentasi data
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3852\2041249354.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF["frame%.0d" % i] = frame # Hasil segmentasi data
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3852\2041249354.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor p